In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\DELL\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
import os

In [4]:
from sklearn.metrics import r2_score,mean_squared_error

In [5]:
os.chdir('../output_data')

In [6]:
df = pd.read_csv('train_v04.csv',header=0)

In [7]:
df.shape

(85799, 218)

In [7]:
os.chdir('../Models')

In [11]:
x = df.drop(['actual_return'],axis=1)

In [12]:
y = np.array(df['actual_return'])

In [14]:
model = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.07, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [15]:
model.fit(x,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.07, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [29]:
pickle.dump(model,open('RF_full.pickle','wb'))

In [16]:
y_pred = model.predict(x)

In [17]:
r2_score(y,y_pred)

0.010094613615247772

In [8]:
os.chdir('../pickles/')

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [13]:
scale = StandardScaler()
scale.fit(x)
pickle.dump(scale,open('scale.pickle','wb'))

In [14]:
x_scaled = scale.transform(x)
pca = PCA(n_components=x_scaled.shape[1])
pca.fit(x_scaled)

PCA(copy=True, iterated_power='auto', n_components=217, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [15]:
var_ratio = np.cumsum(pca.explained_variance_ratio_)
for i in range(x_scaled.shape[1]):
    if var_ratio[i] >=.9:
        n_components=i+1
        break

In [16]:
pca = PCA(n_components=n_components)
pca.fit(x_scaled)
pickle.dump(pca,open('pca.pickle','wb'))

In [42]:
scale = pickle.load(open('scale.pickle','rb'))
pca = pickle.load(open('pca.pickle','rb'))

In [43]:
pca_x = pca.transform(scale.transform(x))

In [44]:
model = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.07, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [45]:
model.fit(pca_x,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=0.07, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=True, random_state=42, verbose=0, warm_start=False)

In [46]:
os.chdir('../Models/')

In [47]:
pickle.dump(model,open('RF_pca.pickle','wb'))

In [48]:
y_pred = model.predict(pca_x)

In [49]:
r2_score(y,y_pred)

0.006150919735918747